#Step 1: Data Cleaning Script
 Goal:


*  Remove duplicates
*  Normalize event participation values
*  Flag missing or incomplete data
*  Save a clean CSV

Actions:
* Remove duplicate email rows
* Convert has_joined_event values from "Yes/No" to True/False

Flag:
* Missing/incomplete LinkedIn profiles
* Blank job titles




In [7]:
import pandas as pd

# Step 1: Load the Excel file
file_path = "/content/Data.xlsx"  # Make sure this file is in the same directory as this script
df = pd.read_excel(file_path)

# Step 2: Remove duplicate email rows
df_cleaned = df.drop_duplicates(subset='email', keep='first')

# Step 3: Normalize 'has_joined_event' values (Yes/No → True/False)
df_cleaned['has_joined_event'] = (
    df_cleaned['has_joined_event']
    .astype(str)
    .str.strip()
    .str.lower()
    .map({'yes': True, 'no': False})
)

# Step 4: Flag rows with missing or incomplete LinkedIn profiles
def is_linkedin_invalid(profile):
    if pd.isna(profile) or not isinstance(profile, str):
        return True
    return not ('linkedin.com/in/' in profile.lower())

df_cleaned['linkedin_invalid'] = df_cleaned['What is your LinkedIn profile?'].apply(is_linkedin_invalid)

# Step 5: Flag rows where the job title is missing or blank
df_cleaned['job_title_missing'] = (
    df_cleaned['Job Title'].isna() |
    (df_cleaned['Job Title'].astype(str).str.strip() == '')
)

# Step 6: Save the cleaned data to a new CSV file
output_file = "cleaned_output.csv"
df_cleaned.to_csv(output_file, index=False)

print(f"Data cleaning complete. File saved as: {output_file}")


Data cleaning complete. File saved as: cleaned_output.csv


<ipython-input-7-5b7c2b17d77d>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['has_joined_event'] = (
<ipython-input-7-5b7c2b17d77d>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['linkedin_invalid'] = df_cleaned['What is your LinkedIn profile?'].apply(is_linkedin_invalid)
<ipython-input-7-5b7c2b17d77d>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

# Step 2: Auto-Personalized Messaging Script
Goal:
* Generate customized messages using name, job title, event status, and LinkedIn info

Logic:
* If joined → thank and promote tool
* If not joined → apologize and offer another session
* If LinkedIn missing → prompt to connect

Output:
* messages_output.csv: contains email, message
* messages_txt/: folder with .txt files for each user

In [8]:
import pandas as pd
import os

# Step 1: Load cleaned data
input_file = "/content/cleaned_output.csv"  # Make sure this file is in the same directory
df = pd.read_csv(input_file)

# Step 2: Define a message generation function
def generate_message(row):
    # Use 'first_name' if present; otherwise fallback to the first word in 'name'
    name = row.get('first_name') if pd.notna(row.get('first_name')) else str(row.get('name')).split()[0]
    job_title = row.get('Job Title') if pd.notna(row.get('Job Title')) else "professional"
    joined = row.get('has_joined_event')
    linkedin_missing = row.get('linkedin_invalid')

    # Customize message
    if joined:
        message = f"Hey {name}, thanks for joining our session! As a {job_title.lower()}, we think you’ll love our upcoming AI workflow tools. Want early access?"
    else:
        message = f"Hi {name}, sorry we missed you at the last event! We’re preparing another session that might better suit your interests as a {job_title.lower()}."

    if linkedin_missing:
        message += " Also, feel free to share your LinkedIn profile so we can stay connected!"

    return message

# Step 3: Apply function and generate message column
df['message'] = df.apply(generate_message, axis=1)

# Step 4: Save all messages to a single CSV file
output_csv = "messages_output.csv"
df[['email', 'message']].to_csv(output_csv, index=False)
print(f" Messages saved to {output_csv}")

# Step 5 (Bonus): Save each message as a separate .txt file
output_txt_folder = "messages_txt"
os.makedirs(output_txt_folder, exist_ok=True)

for _, row in df.iterrows():
    name_part = row.get('first_name') or str(row.get('name')).split()[0]
    email_part = row['email'].split('@')[0]
    filename = f"{name_part}_{email_part}.txt".replace(" ", "_")

    with open(os.path.join(output_txt_folder, filename), "w") as file:
        file.write(row['message'])

print(f"Individual messages saved in the folder: {output_txt_folder}")


 Messages saved to messages_output.csv
Individual messages saved in the folder: messages_txt


# Step 3 (Bonus): Optional Automation Script
Goal:
* Automatically send messages using Email (SMTP) or Telegram Bot

# A. Send Emails using Gmail SMTP

In [4]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import pandas as pd

# Email config
SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 587
SENDER_EMAIL = "your_email@gmail.com"
SENDER_PASSWORD = "your_app_password"  # Use an app password!

# Load messages
df = pd.read_csv("messages_output.csv")

def send_email(recipient_email, message_text):
    msg = MIMEMultipart()
    msg['From'] = SENDER_EMAIL
    msg['To'] = recipient_email
    msg['Subject'] = "Thanks for attending (or missing) our event!"

    msg.attach(MIMEText(message_text, 'plain'))

    with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
        server.starttls()
        server.login(SENDER_EMAIL, SENDER_PASSWORD)
        server.send_message(msg)
        print(f"✅ Email sent to {recipient_email}")

# Loop through and send
for _, row in df.iterrows():
    try:
        send_email(row['email'], row['message'])
    except Exception as e:
        print(f"❌ Failed to send to {row['email']}: {e}")


❌ Failed to send to *****@gmail.com: (535, b'5.7.8 Username and Password not accepted. For more information, go to\n5.7.8  https://support.google.com/mail/?p=BadCredentials 8926c6da1cb9f-4fbcc48c897sm4427665173.98 - gsmtp')
❌ Failed to send to *****@sirmvit.edu: (535, b'5.7.8 Username and Password not accepted. For more information, go to\n5.7.8  https://support.google.com/mail/?p=BadCredentials ca18e2360f4ac-86a26c91a91sm419176039f.28 - gsmtp')
❌ Failed to send to *****@student.embuni.ac.ke: (535, b'5.7.8 Username and Password not accepted. For more information, go to\n5.7.8  https://support.google.com/mail/?p=BadCredentials 8926c6da1cb9f-4fbcc48c4d7sm4398466173.80 - gsmtp')
❌ Failed to send to *****@joinfleek.com: (535, b'5.7.8 Username and Password not accepted. For more information, go to\n5.7.8  https://support.google.com/mail/?p=BadCredentials 8926c6da1cb9f-4fbcc4af616sm4383795173.112 - gsmtp')
❌ Failed to send to *****@simplelogin.co: (535, b'5.7.8 Username and Password not acce

# B. Send via Telegram Bot

In [5]:
import requests
import pandas as pd

BOT_TOKEN = "your_bot_token"
CHAT_ID = "your_chat_id"  # You can get this using @userinfobot

# Load messages
df = pd.read_csv("messages_output.csv")

def send_telegram_message(message_text):
    url = f"https://api.telegram.org/bot{BOT_TOKEN}/sendMessage"
    payload = {
        'chat_id': CHAT_ID,
        'text': message_text
    }
    response = requests.post(url, data=payload)
    if response.status_code == 200:
        print("✅ Sent to Telegram")
    else:
        print(f"❌ Telegram error: {response.text}")

# Send messages in batch
for _, row in df.iterrows():
    try:
        send_telegram_message(row['message'])
    except Exception as e:
        print(f"❌ Failed for {row['email']}: {e}")


❌ Telegram error: {"ok":false,"error_code":404,"description":"Not Found"}
❌ Telegram error: {"ok":false,"error_code":404,"description":"Not Found"}
❌ Telegram error: {"ok":false,"error_code":404,"description":"Not Found"}
❌ Telegram error: {"ok":false,"error_code":404,"description":"Not Found"}
❌ Telegram error: {"ok":false,"error_code":404,"description":"Not Found"}
❌ Telegram error: {"ok":false,"error_code":404,"description":"Not Found"}
❌ Telegram error: {"ok":false,"error_code":404,"description":"Not Found"}
❌ Telegram error: {"ok":false,"error_code":404,"description":"Not Found"}
❌ Telegram error: {"ok":false,"error_code":404,"description":"Not Found"}
❌ Telegram error: {"ok":false,"error_code":404,"description":"Not Found"}
❌ Telegram error: {"ok":false,"error_code":404,"description":"Not Found"}
❌ Telegram error: {"ok":false,"error_code":404,"description":"Not Found"}
❌ Telegram error: {"ok":false,"error_code":404,"description":"Not Found"}
❌ Telegram error: {"ok":false,"error_c